> ## US Drought & Meteorological Data Starter Notebook
This notebook will walk you trough loading the data and create a Dummy Classifier, showing a range of F1 scores that correspond to random predictions if given theclass priors.

## Loading the Data
In this section, we load the training and validation data into numpy arrays and visualize the drought classes and meteorological attributes.

We load the json files for training, validation and testing into the ``files`` dictionary.

In [1]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
sns.set_style('white')

files = {}

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'train' in filename:
            files['train'] = os.path.join(dirname, filename)
        if 'valid' in filename:
            files['valid'] = os.path.join(dirname, filename)
        if 'test' in filename:
            files['test'] = os.path.join(dirname, filename)

The following classes exist, ranging from no drought (``None``), to extreme drought (``D4``).
This could be treated as a regression, ordinal or classification problem, but for now we will treat it as 5 distinct classes.

In [2]:
class2id = {
    'None': 0,
    'D0': 1,
    'D1': 2,
    'D2': 3,
    'D3': 4,
    'D4': 5,
}
id2class = {v: k for k, v in class2id.items()}

Let's also create a dictionary for the meteorological attributes.

In [3]:
valid_dict = json.load(open(files['valid'], 'r'))
_, first = next(iter(valid_dict['root'].items()))
attributes = sorted(first['values'].keys())

id2attr = {i: k for i, k in enumerate(attributes)}
attr2id = {v: k for k, v in id2attr.items()}

del valid_dict
attr2id

{'PRECTOT': 0,
 'PS': 1,
 'QV2M': 2,
 'T2M': 3,
 'T2MDEW': 4,
 'T2MWET': 5,
 'T2M_MAX': 6,
 'T2M_MIN': 7,
 'T2M_RANGE': 8,
 'TS': 9,
 'WS10M': 10,
 'WS10M_MAX': 11,
 'WS10M_MIN': 12,
 'WS10M_RANGE': 13,
 'WS50M': 14,
 'WS50M_MAX': 15,
 'WS50M_MIN': 16,
 'WS50M_RANGE': 17}

Now we'll define a helper method to load the datasets. This just walks through the json and discards the few samples that are corrupted.

In [4]:
# load one of 'train', 'valid' or 'test'
def loadXY(dataset, shuffle=True, random_state=None):
    data_dict = json.load(open(files[dataset], 'r'))
    keys = sorted(list(data_dict['root'].keys()))
    
    if shuffle:
        if random_state is not None:
            np.random.seed(random_state)
        np.random.shuffle(keys)
        
    # float16 should be enough and saves some memory
    X = np.zeros([len(keys), 90, 18], dtype=np.float16)
    y = np.zeros([len(keys)], dtype=np.float16)
    # track how many samples are skipped
    skip_count = 0
    for i, key in tqdm(enumerate(keys), total=len(keys), desc=f'loading {dataset} dataset'):
        sample = data_dict['root'][key]
        input_arr = np.zeros([90, 18])
        try:
            for a, j in attr2id.items():
                input_arr[:,j] = sample['values'][a]
            X[i-skip_count] = input_arr
            y[i-skip_count] = float(class2id[sample['class']])
        except:
            skip_count += 1
    print(f'[{dataset}]: skipped {skip_count} samples ({round(skip_count/len(keys)*100, 3)}%), loaded {len(keys)-skip_count} samples')
    del data_dict
    return X, y

We now load the datasets, this will take a few minutes and use ~8GB of RAM.

In [5]:
X_train, y_train = loadXY('train', random_state=42)
X_valid, y_valid = loadXY('valid', random_state=42)

loading train dataset:   0%|          | 0/118057 [00:00<?, ?it/s]

[train]: skipped 33 samples (0.028%), loaded 118024 samples


loading valid dataset:   0%|          | 0/20729 [00:00<?, ?it/s]

[valid]: skipped 8 samples (0.039%), loaded 20721 samples


## LSTM
Let's train a simple LSTM on the data, treating this as a regression problem.

### Preprocessing and Loading

In [6]:
batch_size_factor = 3
batch_size = 256 * batch_size_factor

In [7]:
from sklearn.preprocessing import RobustScaler

scaler_dict = {}

for attr_id in id2attr.keys():
    scaler_dict[attr_id] = RobustScaler().fit(
        X_train[:,:,attr_id].reshape(-1, 1)
    )
    X_train[:,:,attr_id] = scaler_dict[attr_id].transform(X_train[:,:,attr_id].reshape(-1, 1)).reshape(-1, 90)

In [8]:
for attr_id in id2attr.keys():
    X_valid[:,:,attr_id] = scaler_dict[attr_id].transform(X_valid[:,:,attr_id].reshape(-1, 1)).reshape(-1, 90)

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size, drop_last=True)

In [12]:
valid_data = TensorDataset(torch.tensor(X_valid), torch.tensor(y_valid))
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size, drop_last=True)

### Model

In [29]:
# hyper parameters
lr = 7e-5 * batch_size_factor
output_size = 1
hidden_dim = 512
dropout = 0.1
n_layers = 4
epochs = 10
clip = 5

In [30]:
import torch
from torch import nn

class DroughtNetLSTM(nn.Module):
    def __init__(self, output_size, num_input_features, hidden_dim, n_layers, drop_prob=0.2):
        super(DroughtNetLSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(num_input_features, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.cuda().to(dtype=torch.float32)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
            weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        )
        return hidden

In [31]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print('using GPU')
else:
    device = torch.device("cpu")
    print('using CPU')


model = DroughtNetLSTM(output_size, len(id2attr), hidden_dim, n_layers, dropout)
model.to(device)

using GPU


DroughtNetLSTM(
  (lstm): LSTM(18, 512, num_layers=4, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [32]:
loss_function = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(train_loader), epochs=epochs)

In [33]:
from sklearn.metrics import f1_score

counter = 0
valid_loss_min = np.Inf

torch.manual_seed(42)
np.random.seed(42)

model.train()

for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for k, (inputs, labels) in tqdm(enumerate(train_loader), desc=f'epoch {i+1}/{epochs}', total=len(train_loader)):
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_function(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        scheduler.step()
        
        if k == len(train_loader) - 1 or k == (len(train_loader) - 1) // 2:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            labels = []
            preds = []
            for inp, lab in valid_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = loss_function(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                for l in lab:
                    labels.append(int(l))
                for p in out.round():
                    if p > 5:
                        p = 5
                    if p < 0:
                        p = 0
                    preds.append(int(p))
            
            # log data
            log_dict = {
                'loss': float(loss),
                'epoch': counter/len(train_loader),
                'step': counter,
                'lr': scheduler.get_last_lr()[0]
            }
            log_dict['validation_loss'] = np.mean(val_losses)
            log_dict[f'macro_f1'] = f1_score(labels, preds, average='macro')
            log_dict[f'micro_f1'] = f1_score(labels, preds, average='micro')
            for j, f1 in enumerate(f1_score(labels, preds, average=None)):
                log_dict[f'{id2class[j]}_f1'] = f1
            print(log_dict)
            
            model.train()
            
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

epoch 1/10:   0%|          | 0/153 [00:00<?, ?it/s]

{'loss': 1.4948973655700684, 'epoch': 0.5032679738562091, 'step': 77, 'lr': 2.2136026153986765e-05, 'validation_loss': 1.9406463549687312, 'macro_f1': 0.13311073007851276, 'micro_f1': 0.6648137019230769, 'None_f1': 0.7986643804710766, 'D0_f1': 0.0, 'D1_f1': 0.0, 'D2_f1': 0.0, 'D3_f1': 0.0, 'D4_f1': 0.0}
Validation loss decreased (inf --> 1.940646).  Saving model ...
{'loss': 1.2560970783233643, 'epoch': 1.0, 'step': 153, 'lr': 5.899972870408802e-05, 'validation_loss': 1.4414000786267793, 'macro_f1': 0.15413493156166938, 'micro_f1': 0.425380608974359, 'None_f1': 0.6179924242424242, 'D0_f1': 0.28008118295158013, 'D1_f1': 0.026735982176011884, 'D2_f1': 0.0, 'D3_f1': 0.0, 'D4_f1': 0.0}
Validation loss decreased (1.940646 --> 1.441400).  Saving model ...


epoch 2/10:   0%|          | 0/153 [00:00<?, ?it/s]

{'loss': 0.9294934272766113, 'epoch': 1.5032679738562091, 'step': 230, 'lr': 0.00010989141933666985, 'validation_loss': 1.0183692574501038, 'macro_f1': 0.2079546303216824, 'micro_f1': 0.5318008814102564, 'None_f1': 0.7476325958639433, 'D0_f1': 0.2911502500543596, 'D1_f1': 0.20196413321947054, 'D2_f1': 0.006980802792321117, 'D3_f1': 0.0, 'D4_f1': 0.0}
Validation loss decreased (1.441400 --> 1.018369).  Saving model ...
{'loss': 1.0269651412963867, 'epoch': 2.0, 'step': 306, 'lr': 0.00015999866590906423, 'validation_loss': 1.0108393453634703, 'macro_f1': 0.21708474616889575, 'micro_f1': 0.5995092147435898, 'None_f1': 0.8130984749970446, 'D0_f1': 0.3260082105771553, 'D1_f1': 0.15592515592515593, 'D2_f1': 0.007476635514018692, 'D3_f1': 0.0, 'D4_f1': 0.0}
Validation loss decreased (1.018369 --> 1.010839).  Saving model ...


epoch 3/10:   0%|          | 0/153 [00:00<?, ?it/s]

...

{'loss': 0.7290555834770203, 'epoch': 7.503267973856209, 'step': 1148, 'lr': 5.9027006983178876e-05, 'validation_loss': 0.7990255493384141, 'macro_f1': 0.28880992179900933, 'micro_f1': 0.5863882211538461, 'None_f1': 0.7965077233042311, 'D0_f1': 0.34230679972151307, 'D1_f1': 0.2428953121204761, 'D2_f1': 0.1879656160458453, 'D3_f1': 0.16318407960199005, 'D4_f1': 0.0}
{'loss': 0.7926998138427734, 'epoch': 8.0, 'step': 1224, 'lr': 3.9293732094074813e-05, 'validation_loss': 0.8101752262849075, 'macro_f1': 0.2854590612728804, 'micro_f1': 0.6034154647435898, 'None_f1': 0.8116552003237557, 'D0_f1': 0.3301044156497673, 'D1_f1': 0.24546370967741932, 'D2_f1': 0.1727962638645651, 'D3_f1': 0.15273477812177502, 'D4_f1': 0.0}


epoch 9/10:   0%|          | 0/153 [00:00<?, ?it/s]

{'loss': 0.7214528918266296, 'epoch': 8.50326797385621, 'step': 1301, 'lr': 2.2621187822196478e-05, 'validation_loss': 0.7991858629079965, 'macro_f1': 0.28605553177384535, 'micro_f1': 0.5848357371794872, 'None_f1': 0.7959124087591241, 'D0_f1': 0.327277102635062, 'D1_f1': 0.2528301886792452, 'D2_f1': 0.18330414477524812, 'D3_f1': 0.15700934579439252, 'D4_f1': 0.0}
{'loss': 0.779788613319397, 'epoch': 9.0, 'step': 1377, 'lr': 1.0265839118053297e-05, 'validation_loss': 0.7779870904408969, 'macro_f1': 0.2942362025876522, 'micro_f1': 0.5939503205128205, 'None_f1': 0.8038987971795936, 'D0_f1': 0.33398580866161004, 'D1_f1': 0.2567149988115046, 'D2_f1': 0.19329164297896534, 'D3_f1': 0.17752596789423986, 'D4_f1': 0.0}


epoch 10/10:   0%|          | 0/153 [00:00<?, ?it/s]

{'loss': 0.7054564356803894, 'epoch': 9.50326797385621, 'step': 1454, 'lr': 2.531585909008714e-06, 'validation_loss': 0.7585876148480636, 'macro_f1': 0.2951007256245468, 'micro_f1': 0.6006610576923077, 'None_f1': 0.8089591567852438, 'D0_f1': 0.33775075987841946, 'D1_f1': 0.2632771205638057, 'D2_f1': 0.19152640742890306, 'D3_f1': 0.16909090909090907, 'D4_f1': 0.0}
{'loss': 0.7762449979782104, 'epoch': 10.0, 'step': 1530, 'lr': 1.2917290898069753e-09, 'validation_loss': 0.7651898906781123, 'macro_f1': 0.29370493271259696, 'micro_f1': 0.5978565705128205, 'None_f1': 0.8069972770030529, 'D0_f1': 0.33548387096774196, 'D1_f1': 0.25910329452563785, 'D2_f1': 0.189424753050552, 'D3_f1': 0.17122040072859745, 'D4_f1': 0.0}


Best Macro F1 - **0.304**